--Download library--

In [ ]:
!pip install corextopic

--import dependencies--

In [1]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import en_core_web_sm
# import en_core_web_lg

from pprint import pprint

## Setup NLP Pipeline and Data Cleaning 

In [2]:
##define nlp and start cleaning data
nlp= en_core_web_sm.load()

# List of stop words to equalize data
stop_list = ["Depression","depression",  "anxiety", "Anxiety"]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True


In [3]:
#clean data

def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

## import depression data

In [132]:
doc_dep = pd.read_csv(r'Datasets/2020_March_r_Depression.csv')
doc_dep = doc_dep[doc_dep.Body != '[removed]']
doc_dep = doc_dep.sample(n=7154) 

In [198]:
doc_dep.shape

(7154, 6)

In [301]:
doc_list = []
def nlp_coverter(body_text):
    return nlp(str(body_text))

In [304]:
for row in doc_dep.itertuples():
    doc_list.append(nlp_coverter(getattr(row, 'Body')))

In [305]:
#However doc_list gives us each post as a list, with individual words being elements 
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc_dep = pd.DataFrame(empt,columns = ['Body'])
#df_doc_dep is a dataframe that has cleaned posts from Depression subreddit.
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)


df_doc_dep['label'] = 0

In [306]:
df_doc_dep

,Body,label
0,text know depressed tough time texte day overr...,0
1,honestly read thing problem major lot shit nee...,0
2,want teacher friend sibling parent wrap arm ok...,0
3,fact self harm way punish 1yr pesky razor blee...,0
4,fucking great shitty house stay day watch f...,0
...,...,...
7149,nan,0
7150,start today college class online mandatory cla...,0
7151,today sad \n sad \n little sad \n sad \n\n ...,0
7152,avoid people lot especially friend completely ...,0


--read in data--

## anxietydataset

In [307]:

doc_anx = pd.read_csv(r'2020_March_r_Anxiety.csv')
doc_anx = doc_anx[doc_anx.Body != '[removed]']
# text_doc_anx = doc_anx['Body'].tolist()

In [308]:
for row in doc_anx.itertuples():
    doc_list_anx.append(nlp_coverter(getattr(row, 'Body')))

In [87]:
# doc_list_anx = []
# # Iterates through each article in the corpus.
# for doc in text_doc_anx:
#     # Passes that article through the pipeline and adds to a new list.
#     pr = nlp(str(doc))
#     doc_list_anx.append(pr)

## from tokens create document list again

In [309]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list_anx:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc_anx = pd.DataFrame(empt,columns = ['Body'])
#df_doc_anx is a dataframe that has cleaned posts from Anxiety subreddit
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)

df_doc_anx['label']=1
df_doc_anx

,Body,label
0,thing anxious focus mean study like maybe lazy...,1
1,know midnight know homework know school know X...,1
2,med euphoric feeling consider narcotic,1
3,heart feel sense yesterday continually worried...,1
4,fear swallow small bone eat chicken sure bite ...,1
...,...,...
14303,want sell ps4 use anymore want money hard pers...,1
14304,yesterday(this morning sleep midnight 12 wake ...,1
14305,bath new house cause flood basement live boyfr...,1
14306,mean find self cross leg,1


## Merge Depression and Anxiety subreddits 

In [310]:
df_col_merged =pd.concat([df_doc_dep, df_doc_anx], axis=0).reset_index(drop=True)
df_col_merged

,Body,label
0,text know depressed tough time texte day overr...,0
1,honestly read thing problem major lot shit nee...,0
2,want teacher friend sibling parent wrap arm ok...,0
3,fact self harm way punish 1yr pesky razor blee...,0
4,fucking great shitty house stay day watch f...,0
...,...,...
21457,want sell ps4 use anymore want money hard pers...,1
21458,yesterday(this morning sleep midnight 12 wake ...,1
21459,bath new house cause flood basement live boyfr...,1
21460,mean find self cross leg,1


## Obtain TF-IDF score for combined dataframes

In [353]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.2,
    min_df=8,
    max_features=None,
    norm=None,
    binary=True,
    use_idf=False,
)

tfidf = vectorizer.fit_transform(df_col_merged.Body)
features = vectorizer.get_feature_names()
print(len(vocab))

6414


In [354]:
print(tfidf[1])

  (0, 3478)	1.0
  (0, 6080)	1.0
  (0, 4826)	1.0
  (0, 2404)	1.0
  (0, 3306)	1.0
  (0, 2772)	1.0
  (0, 4588)	1.0
  (0, 4398)	1.0
  (0, 3429)	1.0
  (0, 5095)	1.0
  (0, 3798)	1.0
  (0, 1279)	1.0
  (0, 2194)	1.0
  (0, 3187)	1.0
  (0, 5716)	1.0
  (0, 924)	1.0
  (0, 2593)	1.0
  (0, 2766)	1.0
  (0, 4087)	1.0
  (0, 1403)	1.0
  (0, 1399)	1.0
  (0, 301)	1.0
  (0, 4183)	1.0
  (0, 3691)	1.0
  (0, 3533)	1.0
  :	:
  (0, 3464)	1.0
  (0, 2097)	1.0
  (0, 5792)	1.0
  (0, 4153)	1.0
  (0, 2359)	1.0
  (0, 5664)	1.0
  (0, 1200)	1.0
  (0, 6169)	1.0
  (0, 2564)	1.0
  (0, 3346)	1.0
  (0, 2884)	1.0
  (0, 5321)	1.0
  (0, 3014)	1.0
  (0, 2645)	1.0
  (0, 847)	1.0
  (0, 635)	1.0
  (0, 6)	1.0
  (0, 4901)	1.0
  (0, 1991)	1.0
  (0, 3413)	1.0
  (0, 4942)	1.0
  (0, 2648)	1.0
  (0, 1540)	1.0
  (0, 2658)	1.0
  (0, 3386)	1.0


In [315]:
from sklearn.feature_selection import SelectKBest
tfidf_reduced = SelectKBest(k=100).fit_transform(tfidf, df_col_merged.Body)

D:\Anaconda\envs\python37\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [317]:
for i in tfidf_reduced:
    print(i)



  (0, 2)	1.0
  (0, 45)	1.0
  (0, 53)	1.0
  (0, 74)	1.0
  (0, 46)	1.0
  (0, 49)	1.0
  (0, 27)	1.0


  (0, 2)	1.0
  (0, 78)	1.0
  (0, 79)	1.0


  (0, 2)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 78)	1.0



  (0, 2)	1.0
  (0, 2)	1.0
  (0, 35)	1.0


  (0, 53)	1.0
  (0, 43)	1.0
  (0, 46)	1.0





  (0, 16)	1.0
  (0, 53)	1.0
  (0, 27)	1.0

  (0, 39)	1.0
  (0, 19)	1.0
  (0, 71)	1.0
  (0, 79)	1.0
  (0, 50)	1.0
  (0, 52)	1.0

  (0, 55)	1.0
  (0, 19)	1.0
  (0, 38)	1.0
  (0, 53)	1.0
  (0, 56)	1.0

  (0, 78)	1.0


  (0, 15)	1.0
  (0, 19)	1.0
  (0, 21)	1.0
  (0, 49)	1.0
  (0, 71)	1.0
  (0, 78)	1.0
  (0, 79)	1.0
  (0, 84)	1.0

  (0, 55)	1.0
  (0, 88)	1.0
  (0, 88)	1.0


  (0, 15)	1.0
  (0, 78)	1.0








  (0, 50)	1.0

  (0, 49)	1.0
  (0, 2)	1.0
  (0, 50)	1.0
  (0, 2)	1.0



  (0, 50)	1.0

  (0, 2)	1.0
  (0, 19)	1.0
  (0, 50)	1.0

  (0, 88)	1.0


  (0, 53)	1.0
  (0, 79)	1.0
  (0, 38)	1.0



  (0, 2)	1.0
  (0, 32)	1.0
  (0, 38)	1.0

  (0, 2)	1.0
  (0, 53)	1.0
  (0, 2)	1.0
  (0, 27)	1.0
  (0, 32)	1.0
 



  (0, 19)	1.0
  (0, 39)	1.0
  (0, 84)	1.0

  (0, 2)	1.0
  (0, 53)	1.0
  (0, 78)	1.0

  (0, 55)	1.0

  (0, 21)	1.0
  (0, 32)	1.0
  (0, 2)	1.0
  (0, 19)	1.0
  (0, 27)	1.0
  (0, 39)	1.0
  (0, 42)	1.0
  (0, 49)	1.0
  (0, 53)	1.0
  (0, 64)	1.0
  (0, 74)	1.0
  (0, 78)	1.0
  (0, 84)	1.0
  (0, 53)	1.0
  (0, 15)	1.0
  (0, 95)	1.0


  (0, 2)	1.0


  (0, 31)	1.0
  (0, 50)	1.0
  (0, 84)	1.0

  (0, 2)	1.0
  (0, 19)	1.0



  (0, 2)	1.0
  (0, 27)	1.0

  (0, 2)	1.0
  (0, 32)	1.0
  (0, 15)	1.0
  (0, 38)	1.0
  (0, 31)	1.0
  (0, 71)	1.0
  (0, 27)	1.0

  (0, 15)	1.0
  (0, 79)	1.0


  (0, 1)	1.0
  (0, 31)	1.0
  (0, 38)	1.0
  (0, 84)	1.0


  (0, 39)	1.0
  (0, 2)	1.0
  (0, 32)	1.0
  (0, 50)	1.0
  (0, 84)	1.0

  (0, 3)	1.0
  (0, 22)	1.0
  (0, 50)	1.0
  (0, 2)	1.0
  (0, 27)	1.0
  (0, 59)	1.0
  (0, 74)	1.0
  (0, 27)	1.0

  (0, 2)	1.0
  (0, 39)	1.0
  (0, 74)	1.0


  (0, 2)	1.0
  (0, 14)	1.0
  (0, 27)	1.0
  (0, 53)	1.0


  (0, 19)	1.0

  (0, 2)	1.0
  (0, 7)	1.0
  (0, 38)	1.0
  (0, 96)	1.0





  (0, 31)	1.0

  


  (0, 14)	1.0
  (0, 50)	1.0
  (0, 56)	1.0

  (0, 19)	1.0


  (0, 31)	1.0


  (0, 88)	1.0
  (0, 27)	1.0
  (0, 44)	1.0



  (0, 2)	1.0
  (0, 27)	1.0


  (0, 98)	1.0


  (0, 31)	1.0
  (0, 49)	1.0

  (0, 50)	1.0
  (0, 2)	1.0

  (0, 2)	1.0






  (0, 57)	1.0
  (0, 44)	1.0
  (0, 53)	1.0
  (0, 71)	1.0
  (0, 27)	1.0

  (0, 2)	1.0
  (0, 84)	1.0





  (0, 2)	1.0
  (0, 53)	1.0

  (0, 55)	1.0
  (0, 2)	1.0
  (0, 84)	1.0
  (0, 98)	1.0
  (0, 33)	1.0
  (0, 15)	1.0
  (0, 27)	1.0







  (0, 50)	1.0


  (0, 32)	1.0
  (0, 24)	1.0

  (0, 79)	1.0


  (0, 7)	1.0
  (0, 31)	1.0

  (0, 27)	1.0
  (0, 2)	1.0
  (0, 19)	1.0
  (0, 32)	1.0
  (0, 38)	1.0
  (0, 53)	1.0
  (0, 78)	1.0
  (0, 84)	1.0
  (0, 88)	1.0

  (0, 27)	1.0

  (0, 15)	1.0
  (0, 27)	1.0
  (0, 44)	1.0
  (0, 84)	1.0


  (0, 53)	1.0
  (0, 98)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 27)	1.0

  (0, 16)	1.0





  (0, 2)	1.0
  (0, 19)	1.0
  (0, 21)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 67)	1.0
  (0, 79)	1.0

  (0, 53)	1.0

  (0, 88)	1.0
  (0, 50)	1.0
  (0

  (0, 2)	1.0

  (0, 21)	1.0

  (0, 27)	1.0
  (0, 38)	1.0
  (0, 50)	1.0


  (0, 16)	1.0
  (0, 53)	1.0

  (0, 50)	1.0
  (0, 15)	1.0

  (0, 49)	1.0


  (0, 32)	1.0
  (0, 19)	1.0
  (0, 39)	1.0

  (0, 39)	1.0

  (0, 2)	1.0

  (0, 35)	1.0

  (0, 15)	1.0
  (0, 84)	1.0


  (0, 71)	1.0




  (0, 32)	1.0
  (0, 15)	1.0
  (0, 27)	1.0
  (0, 95)	1.0
  (0, 38)	1.0

  (0, 57)	1.0






  (0, 2)	1.0

  (0, 7)	1.0
  (0, 15)	1.0
  (0, 79)	1.0


  (0, 27)	1.0

  (0, 55)	1.0
  (0, 2)	1.0
  (0, 15)	1.0
  (0, 39)	1.0


  (0, 50)	1.0
  (0, 56)	1.0
  (0, 84)	1.0
  (0, 98)	1.0
  (0, 49)	1.0
  (0, 56)	1.0




  (0, 38)	1.0
  (0, 78)	1.0
  (0, 53)	1.0



  (0, 31)	1.0
  (0, 2)	1.0
  (0, 53)	1.0








  (0, 27)	1.0




  (0, 25)	1.0
  (0, 38)	1.0

  (0, 2)	1.0
  (0, 27)	1.0
  (0, 42)	1.0


  (0, 27)	1.0
  (0, 1)	1.0




  (0, 19)	1.0



  (0, 32)	1.0
  (0, 21)	1.0



  (0, 71)	1.0










  (0, 15)	1.0
  (0, 53)	1.0

  (0, 2)	1.0

  (0, 38)	1.0

  (0, 31)	1.0
  (0, 50)	1.0


  (0, 2)	1.0
  (0, 14)	1.0
  (0, 42



  (0, 25)	1.0


  (0, 53)	1.0
  (0, 15)	1.0
  (0, 78)	1.0
  (0, 14)	1.0
  (0, 2)	1.0
  (0, 27)	1.0
  (0, 53)	1.0
  (0, 90)	1.0
  (0, 91)	1.0
  (0, 95)	1.0
  (0, 2)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 38)	1.0



  (0, 2)	1.0


  (0, 38)	1.0

  (0, 27)	1.0

  (0, 2)	1.0
  (0, 19)	1.0
  (0, 27)	1.0







  (0, 39)	1.0









  (0, 27)	1.0

  (0, 2)	1.0




  (0, 78)	1.0

  (0, 71)	1.0

  (0, 1)	1.0
  (0, 7)	1.0
  (0, 11)	1.0
  (0, 14)	1.0
  (0, 27)	1.0
  (0, 53)	1.0
  (0, 83)	1.0
  (0, 84)	1.0
  (0, 31)	1.0

  (0, 2)	1.0
  (0, 64)	1.0
  (0, 14)	1.0
  (0, 19)	1.0
  (0, 38)	1.0
  (0, 56)	1.0



  (0, 2)	1.0
  (0, 19)	1.0
  (0, 53)	1.0
  (0, 79)	1.0


  (0, 38)	1.0
  (0, 46)	1.0
  (0, 84)	1.0
  (0, 15)	1.0
  (0, 2)	1.0
  (0, 53)	1.0

  (0, 50)	1.0
  (0, 53)	1.0
  (0, 54)	1.0

  (0, 2)	1.0
  (0, 16)	1.0
  (0, 38)	1.0
  (0, 50)	1.0
  (0, 53)	1.0
  (0, 22)	1.0
  (0, 2)	1.0

  (0, 15)	1.0
  (0, 19)	1.0


  (0, 39)	1.0



  (0, 27)	1.0
  (0, 6)	1.0
  (0, 50)	1.0
  (0, 53)	1.0
  (0, 54)	1.0

...

  (0, 31)	1.0
  (0, 75)	1.0
  (0, 2)	1.0

  (0, 15)	1.0

  (0, 35)	1.0


  (0, 53)	1.0
  (0, 15)	1.0
  (0, 27)	1.0
  (0, 53)	1.0
  (0, 84)	1.0
  (0, 79)	1.0


  (0, 27)	1.0
  (0, 27)	1.0
  (0, 2)	1.0





  (0, 34)	1.0
  (0, 54)	1.0
  (0, 55)	1.0
  (0, 95)	1.0


  (0, 78)	1.0
  (0, 79)	1.0
  (0, 24)	1.0
  (0, 38)	1.0
  (0, 79)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 50)	1.0
  (0, 46)	1.0
  (0, 79)	1.0
  (0, 15)	1.0
  (0, 56)	1.0
  (0, 53)	1.0
  (0, 54)	1.0
  (0, 79)	1.0

  (0, 57)	1.0
  (0, 27)	1.0



  (0, 1)	1.0
  (0, 32)	1.0
  (0, 56)	1.0

  (0, 50)	1.0
  (0, 50)	1.0


  (0, 59)	1.0
  (0, 2)	1.0
  (0, 32)	1.0
  (0, 27)	1.0

  (0, 2)	1.0
  (0, 15)	1.0
  (0, 27)	1.0
  (0, 53)	1.0
  (0, 27)	1.0

  (0, 87)	1.0


  (0, 19)	1.0
  (0, 22)	1.0






  (0, 53)	1.0
  (0, 15)	1.0
  (0, 32)	1.0
  (0, 98)	1.0



  (0, 2)	1.0
  (0, 19)	1.0
  (0, 32)	1.0
  (0, 31)	1.0

  (0, 6)	1.0
  (0, 55)	1.0

  (0, 27)	1.0
  (0, 34)	1.0
  (0, 50)	1.0
  (0, 71)	1.0
  (0, 45)	1.0
  (0, 84)	1.0
  (0, 98)	1.0

  (0,

  (0, 96)	1.0



  (0, 31)	1.0
  (0, 33)	1.0

  (0, 53)	1.0




  (0, 2)	1.0
  (0, 32)	1.0
  (0, 53)	1.0
  (0, 21)	1.0
  (0, 32)	1.0
  (0, 57)	1.0
  (0, 15)	1.0






  (0, 56)	1.0

  (0, 64)	1.0




  (0, 31)	1.0
  (0, 27)	1.0
  (0, 38)	1.0
  (0, 27)	1.0
  (0, 19)	1.0
  (0, 53)	1.0
  (0, 2)	1.0
  (0, 32)	1.0
  (0, 2)	1.0
  (0, 27)	1.0
  (0, 84)	1.0
  (0, 32)	1.0
  (0, 53)	1.0

  (0, 27)	1.0
  (0, 32)	1.0
  (0, 65)	1.0
  (0, 31)	1.0
  (0, 38)	1.0
  (0, 57)	1.0
  (0, 31)	1.0
  (0, 35)	1.0
  (0, 84)	1.0
  (0, 14)	1.0
  (0, 27)	1.0
  (0, 56)	1.0





  (0, 53)	1.0


  (0, 27)	1.0
  (0, 37)	1.0



  (0, 32)	1.0

  (0, 2)	1.0
  (0, 31)	1.0
  (0, 38)	1.0
  (0, 54)	1.0
  (0, 65)	1.0

  (0, 21)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 46)	1.0
  (0, 50)	1.0
  (0, 53)	1.0
  (0, 71)	1.0
  (0, 74)	1.0
  (0, 79)	1.0
  (0, 31)	1.0
  (0, 78)	1.0
  (0, 21)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 46)	1.0
  (0, 50)	1.0
  (0, 53)	1.0
  (0, 71)	1.0
  (0, 74)	1.0
  (0, 79)	1.0
  (0, 53)	1.0
  (0, 79)	1.0











  (0, 15)	1.0
  (0, 32)	1.0


  (0, 2)	1.0
  (0, 19)	1.0
  (0, 80)	1.0
  (0, 31)	1.0
  (0, 15)	1.0
  (0, 19)	1.0
  (0, 31)	1.0



  (0, 31)	1.0
  (0, 84)	1.0








  (0, 83)	1.0

  (0, 22)	1.0
  (0, 71)	1.0
  (0, 31)	1.0

  (0, 53)	1.0
  (0, 84)	1.0
  (0, 15)	1.0
  (0, 46)	1.0
  (0, 54)	1.0



  (0, 31)	1.0
  (0, 32)	1.0
  (0, 78)	1.0
  (0, 19)	1.0
  (0, 50)	1.0
  (0, 56)	1.0
  (0, 31)	1.0
  (0, 32)	1.0




  (0, 15)	1.0

  (0, 2)	1.0

  (0, 27)	1.0

  (0, 31)	1.0
  (0, 84)	1.0
  (0, 27)	1.0

  (0, 31)	1.0
  (0, 32)	1.0
  (0, 84)	1.0
  (0, 55)	1.0





  (0, 15)	1.0
  (0, 71)	1.0
  (0, 81)	1.0

  (0, 1)	1.0
  (0, 32)	1.0
  (0, 56)	1.0
  (0, 31)	1.0
  (0, 31)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 78)	1.0



  (0, 2)	1.0


  (0, 15)	1.0
  (0, 19)	1.0
  (0, 27)	1.0





  (0, 2)	1.0
  (0, 14)	1.0
  (0, 27)	1.0
  (0, 84)	1.0
  (0, 27)	1.0
  (0, 32)	1.0
  (0, 84)	1.0


  (0, 23)	1.0




  (0, 15)	1.0
  (0, 98)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 84)	1.0
  (0, 97)	1.0
  (0, 15)	1.0
  






  (0, 19)	1.0
  (0, 32)	1.0
  (0, 33)	1.0
  (0, 50)	1.0
  (0, 71)	1.0
  (0, 93)	1.0

  (0, 2)	1.0
  (0, 54)	1.0


  (0, 31)	1.0








  (0, 23)	1.0

  (0, 22)	1.0
  (0, 27)	1.0
  (0, 31)	1.0
  (0, 71)	1.0


  (0, 27)	1.0


  (0, 31)	1.0
  (0, 56)	1.0

  (0, 15)	1.0

  (0, 64)	1.0
  (0, 66)	1.0
  (0, 27)	1.0
  (0, 31)	1.0


  (0, 31)	1.0
  (0, 32)	1.0
  (0, 36)	1.0



  (0, 32)	1.0
  (0, 53)	1.0


  (0, 46)	1.0
  (0, 96)	1.0


  (0, 32)	1.0

  (0, 15)	1.0
  (0, 27)	1.0
  (0, 35)	1.0
  (0, 96)	1.0
  (0, 57)	1.0




  (0, 32)	1.0
  (0, 79)	1.0

  (0, 53)	1.0
  (0, 79)	1.0




  (0, 64)	1.0
  (0, 79)	1.0

  (0, 27)	1.0
  (0, 50)	1.0
  (0, 50)	1.0

  (0, 79)	1.0


  (0, 2)	1.0
  (0, 53)	1.0
  (0, 84)	1.0
  (0, 2)	1.0
  (0, 53)	1.0
  (0, 84)	1.0
  (0, 24)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 53)	1.0
  (0, 32)	1.0

  (0, 79)	1.0
  (0, 2)	1.0





  (0, 78)	1.0
  (0, 79)	1.0



  (0, 33)	1.0




  (0, 2)	1.0
  (0, 27)	1.0
  (0, 32)	1.0
  (0, 55)	1.0



  (0, 2)	1.0
  (0, 32)	1.0


  (0


  (0, 2)	1.0
  (0, 29)	1.0
  (0, 71)	1.0
  (0, 80)	1.0

  (0, 15)	1.0
  (0, 27)	1.0
  (0, 19)	1.0

  (0, 32)	1.0


  (0, 53)	1.0
  (0, 74)	1.0
  (0, 19)	1.0
  (0, 50)	1.0
  (0, 52)	1.0
  (0, 53)	1.0

  (0, 37)	1.0
  (0, 84)	1.0

  (0, 6)	1.0
  (0, 50)	1.0

  (0, 15)	1.0

  (0, 53)	1.0
  (0, 2)	1.0
  (0, 84)	1.0
  (0, 24)	1.0

  (0, 84)	1.0
  (0, 90)	1.0
  (0, 96)	1.0
  (0, 78)	1.0



  (0, 15)	1.0
  (0, 31)	1.0

  (0, 2)	1.0
  (0, 38)	1.0
  (0, 53)	1.0
  (0, 88)	1.0


  (0, 32)	1.0




  (0, 7)	1.0
  (0, 15)	1.0
  (0, 32)	1.0





  (0, 27)	1.0
  (0, 32)	1.0
  (0, 38)	1.0
  (0, 2)	1.0
  (0, 53)	1.0
  (0, 78)	1.0
  (0, 27)	1.0
  (0, 50)	1.0
  (0, 83)	1.0
  (0, 84)	1.0
  (0, 95)	1.0
  (0, 21)	1.0
  (0, 31)	1.0

  (0, 32)	1.0
  (0, 56)	1.0

  (0, 50)	1.0
  (0, 53)	1.0

  (0, 27)	1.0
  (0, 53)	1.0
  (0, 79)	1.0

  (0, 62)	1.0
  (0, 31)	1.0
  (0, 2)	1.0
  (0, 21)	1.0
  (0, 50)	1.0
  (0, 79)	1.0
  (0, 23)	1.0

  (0, 31)	1.0
  (0, 68)	1.0
  (0, 79)	1.0
  (0, 31)	1.0
  (0, 68)	1.0
  (0, 79)	1

## Machine Learning

In [ ]:
X = tfidf;
y = df_col_merged.label.tolist()

import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)


## clf.score provides the accuracy of machine learning model

In [ ]:
clf.score(X_test,y_test)

## Topic Modelling

In [ ]:
#read in data
doc = pd.read_csv(r'Datasets/2020_March_r_Depression.csv')
doc = doc[doc.Body != '[removed]']

In [ ]:
#create a list of documents (list of lists) 
text_doc = doc['Body'].tolist()

In [ ]:
nlp= spacy.load("en")

# List of stop words to equalize data
stop_list = ["Depression","depression", "coronavirus", "quarantine", "coronavirus", "Coronavirus", "lockdown", "anxiety", "Anxiety", "Quarantine", "Lockdown", "Agoraphobia", "Agoraphobic", "agoraphobic", "agoraphobia"]

# Updates spaCy's default stop words list with my additional words. 
nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [ ]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)



In [ ]:
doc_list = []
# Iterates through each article in the corpus.
for doc in text_doc:
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(str(doc))
    doc_list.append(pr)

In [ ]:
#however doc_list gives us each post as a list, with individual words being elements 
from nltk.tokenize.treebank import TreebankWordDetokenizer
empt = [];
for doc in doc_list:
    a1 = TreebankWordDetokenizer().detokenize(doc)
    empt.append(a1)
    
df_doc = pd.DataFrame(empt,columns = ['Body'])
#df_doc is a dataframe that has cleaned posts. 
#all the 'removed' posts are gone and all the stopwords in the individual posts are gone! 

#both doc_list and df_doc are important (at least I think so :D)


df_doc

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df_doc.Body)
tfidf = vectorizer.transform(df_doc.Body)
vocab = vectorizer.get_feature_names()
print(len(vocab))

In [ ]:
#import corextopic

from corextopic import corextopic as ct
anchors = []
model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [ ]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=15)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ["family"],
    ["die"],
    [""],
    [""],
    [""],
    [""],

]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [ ]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df.iloc[0:10]

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df

## LDA Unsupervised

In [ ]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=6, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
#print the keyword in the 10 topics
pprint(lda_model.print_topics(num_words=100))

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=doc_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=doc_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(100)

In [ ]:
df_dominant_topic.head(25)
#df_dominant_topic.to_csv(r'depression_2020_datest.csv')

In [ ]:
from collections import Counter
Counter(df_dominant_topic.Dominant_Topic)

In [ ]:
newest_doc = newest_doc[newest_doc.Body != '[removed]'].dropna()

In [ ]:
newest_doc.Body

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)

df = pd.concat([df, topic_df], axis=1)

In [ ]:
topic_df